In [2]:
%cd ..

c:\Users\facua\Master in Data Science\ELEC 631\quantized-education-v2


c:\Users\facua\anaconda3\envs\assignment_@\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from commons import model, benchmarks
import numpy as np
import importlib
import torch.nn as nn

c:\Users\facua\anaconda3\envs\assignment_@\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load components

In [4]:
dataset = benchmarks.BenchmarkDataset()

In [8]:
randrange = np.random.random_integers(0, 100, 3)
rows = dataset.test_set.select(randrange)

/var/folders/2c/s1dc7gqn0jz4dls2kprq822m0000gn/T/ipykernel_1502/1691709274.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 100 + 1) instead
  randrange = np.random.random_integers(0, 100, 3)


In [9]:
rows[0]

{'question': 'This question refers to the following information.\nIt is natural, it is a privilege, I will go farther, it is a right, which all free men claim, that they are entitled to complain when they are hurt. They have a right publicly to remonstrate against the abuses of power in the strongest terms, to put their neighbors upon their guard against the craft or open violence of men in authority, and to assert with courage the sense they have of the blessings of liberty, the value they put upon it, and their resolution at all hazards to preserve it as one of the greatest blessings heaven can bestow.… But to conclude: The question before the Court and you, Gentlemen of the jury, is not of small or private concern. It is not the cause of one poor printer, nor of New York alone, which you are now trying. No! It may in its consequence affect every free man that lives under a British government on the main of America. It is the best cause. It is the cause of liberty. And I make no doub

In [12]:
smol = model.SmolModel()

In [53]:
smol.model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm)

# Test inference

In [67]:
# run a sample evaluation on a single row
prompt = f''''
{rows[0]["question"]}

CHOICES:
A) {rows[0]["choices"][0]}
B) {rows[0]["choices"][1]}
C) {rows[0]["choices"][2]}
D) {rows[0]["choices"][3]}

Please select the correct answer by providing only the corresponding letter (A, B, C, or D).
ANSWER:
'''


tok_q = smol.tokenizer(prompt, return_tensors='pt')
# sizes
tok_q['input_ids'].size()

torch.Size([1, 480])

In [68]:
outputs = smol.model(**tok_q.to(smol.device))

In [69]:
outputs.logits.size() # size = (batch size, len of token q, len of model vocab [input dim of embedding layer])

torch.Size([1, 480, 49152])

In [70]:
logprobs = nn.functional.log_softmax(outputs.logits, dim=-1)
logprobs.size()

torch.Size([1, 480, 49152])

In [71]:
# decode with tokenizer
print(smol.tokenizer.decode(outputs.logits.argmax(dim=-1)[0]))



  What is is to the following information.

 is known for in is human fact, it am not to, I is a duty,
 is men men have, to of may entitled to the of they are oppressed,
 are a right to to expressonstrate against the wrongs of power, their government terms, and call the grievances in their guard, the encro and the violence of those in power, to to call their all and rights of have of their rights of liberty. and rights of set upon their, and the determination to all hazards to preserve it. a of the greatest of of has bestow.
 it complain, I right is the court is the is mymen of the jury, is, whether the or private concern, It is not the mere of an set man, nor even his- alone, which you are now trying; No! It may be its consequences affect every fre man that lives under civil free government upon the main of America.
 may the best cause.
 is the cause of liberty.
 I have no doubt but that verdict virtue will day will be only beitle you to the appro and esteem of every fellow citizens

In [73]:
print(prompt)

'
Question: This question refers to the following information.
It is natural, it is a privilege, I will go farther, it is a right, which all free men claim, that they are entitled to complain when they are hurt. They have a right publicly to remonstrate against the abuses of power in the strongest terms, to put their neighbors upon their guard against the craft or open violence of men in authority, and to assert with courage the sense they have of the blessings of liberty, the value they put upon it, and their resolution at all hazards to preserve it as one of the greatest blessings heaven can bestow.… But to conclude: The question before the Court and you, Gentlemen of the jury, is not of small or private concern. It is not the cause of one poor printer, nor of New York alone, which you are now trying. No! It may in its consequence affect every free man that lives under a British government on the main of America. It is the best cause. It is the cause of liberty. And I make no doubt b

In [84]:
choice_tokens = ['A', 'B', 'C', 'D']
choice_ids = [smol.tokenizer.encode(choice, return_tensors='pt', add_special_tokens=False)[0,0] for choice in choice_tokens]

for t, i in zip(choice_tokens, choice_ids):
    print(t, i)


A tensor(49)
B tensor(50)
C tensor(51)
D tensor(52)


# Test benchmarks

In [10]:
importlib.reload(benchmarks)

<module 'commons.benchmarks' from 'c:\\Users\\facua\\Master in Data Science\\ELEC 631\\quantized-education-v2\\commons\\benchmarks.py'>

In [11]:
mmlu = benchmarks.BenchmarkDataset(subtasks=["college_biology", "high_school_biology"], sample_size=100)

In [11]:
mmlu.CHOICES

['A', 'B', 'C', 'D']

In [22]:
import os
print(os.getcwd())

file_path = 'rag\data\Biology2e-WEB_ICOFkGu.pdf'
if os.path.exists(file_path):
    print("File path exists.")
else:
    print("File path does not exist.")


c:\Users\facua\Master in Data Science\ELEC 631\quantized-education-v2
File path exists.


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\facua\AppData\Local\Temp\ipykernel_12544\3965606501.py:4: SyntaxWarning: invalid escape sequence '\d'
  file_path = 'rag\data\Biology2e-WEB_ICOFkGu.pdf'


In [12]:
mmlu.load_model("HuggingFaceTB/SmolLM2-1.7B-Instruct",rag=True, pdf_path="rag\data\Biology2e-WEB_ICOFkGu.pdf")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\facua\AppData\Local\Temp\ipykernel_8328\517071166.py:1: SyntaxWarning: invalid escape sequence '\d'
  mmlu.load_model("HuggingFaceTB/SmolLM2-1.7B-Instruct",rag=True, pdf_path="rag\data\Biology2e-WEB_ICOFkGu.pdf")


Extracted 2787 chunks from the PDF.
RAG pipeline loaded successfully.


In [20]:
mmlu.choice_ids_mapping

{'A': tensor(49), 'B': tensor(50), 'C': tensor(51), 'D': tensor(52)}

In [13]:
acc, subset = mmlu.evaluate()

Evaluating - So far gotten 0 out of 0:   0%|          | 0/100 [00:00<?, ?it/s]

['speciation that can occur when an event causes a small portion of a population to be cut off from the rest of the population reinforcement continued speciation divergence between two related species due to low fitness of hybrids between them reproductive isolation situation that occurs when a species is reproductively independent from other species; behavior, location, or reproductive barriers may cause this to happen speciation formation of a new species species group of populations that interbreed and produce fertile offspring sympatric speciation speciation that occurs in the same geographic space temporal isolation differences in breeding schedules that can act as a form of prezygotic barrier leading to reproductive isolation variation genetic differences among individuals in a population vestigial structure physical structure present in an organism but that has no apparent function and appears to be from a functional structure in a distant ancestor vicariance allopatric speciati

Evaluating - So far gotten 0 out of 0:   1%|          | 1/100 [00:53<1:28:40, 53.74s/it]

['14. Newly-formed root cells begin to form different cell types in the ________. a. zone of elongation b. zone of maturation c. root meristem d. zone of cell division 15. The stalk of a leaf is known as the ________. a. petiole b. lamina c. stipule d. rachis 16. Leaflets are a characteristic of ________ leaves. a. alternate b. whorled c. compound d. opposite 17. Cells of the ________ contain chloroplasts. a. epidermis b. vascular tissue c. stomata d. mesophyll 18. Which of the following is most likely to be found in a desert environment? a. broad leaves to capture sunlight b. spines instead of leaves c. needle-like leaves d. wide, flat leaves that can float 19. When stomata open, what occurs? a. Water vapor is lost to the external environment, increasing the rate of transpiration. b. Water vapor is lost to the external environment, decreasing the rate of transpiration. c. Water vapor enters the spaces in the mesophyll, increasing the rate of transpiration. d. Water vapor enters the sp

Evaluating - So far gotten 0 out of 0:   1%|          | 1/100 [01:35<2:37:10, 95.26s/it]


KeyboardInterrupt: 

In [29]:
mmlu.test_set[0]

{'question': 'All of the following are examples of events that can prevent interspecific breeding EXCEPT',
 'subject': 'high_school_biology',
 'choices': ['the potential mates experience geographic isolation',
  'the potential mates experience behavioral isolation',
  'the potential mates have different courtship rituals',
  'the potential mates have similar breeding seasons'],
 'answer': 3}

In [1]:
acc

NameError: name 'acc' is not defined